## Data ingential trial runs


In [71]:
import os 

In [72]:
os.chdir(r"C:\Users\USER\Desktop\MLDefaults\NLP-Text-Summerizer")

## Entity Trial-runs 

In [73]:
#Defining code for our entity. An entity is a return type of a function 
from dataclasses import dataclass
from pathlib import Path 

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_url: str
    local_data_file: str
    unzip_dir: str

## ConfigurationManager Trial-runs 

In [74]:
#Code for Cinfiguration manager
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml, create_directories 

In [75]:
#Updating the configuration file 
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH, 
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
       
        create_directories([self.config.artifacts_root])
    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file, 
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

## Componets Module Trail-runs

In [76]:
#Importing dependencies 
import os 
from urllib import request 
import zipfile 
from textSummerizer.logging import logger
from textSummerizer.utils.common import get_size

In [77]:
class DataIngestion: 
    def __init__(self, config: DataIngestionConfig):
        self.config=config

    def download_file(self): 
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_url, 
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} downloaded with the following \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)


In [79]:
#Pipeline 
try:
    config =ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-05-26 12:07:48,001: INFO: common: yaml_file: config\config.yaml loaded successfully]
[2025-05-26 12:07:48,036: INFO: common: yaml_file: params.yaml loaded successfully]
[2025-05-26 12:07:48,047: INFO: common: Created directory at: artifacts]
[2025-05-26 12:07:48,048: INFO: common: Created directory at: artifacts/data_ingestion]
[2025-05-26 12:07:48,060: INFO: 1357720767: File already exists of size: ~8181 KB]
